# Data Visualization and Metrics on the CelebA_HQ Dataset

In [1]:
import pandas as pd
import sklearn as sns
import numpy as np
import matplotlib.pyplot as plt

## Populate Dataframes

Each dataframe consists of 5 columns: `image`, `image_path`, `actual_label`, `predicted_label`, `confidence`
* `image` is the image name, for example 10.jpg
* `image_path` is the path to the image
* `actual_label` is the actual label for that person (the original image)
* `predicted_label` is the result of running the original, attacked, or defended image into the classifier. This is the label used to determine if the attack or defense was effective.
* `confidence` is the confidence in which we think that the image belongs to that class. This is the max value of the prediction.

In [68]:
fgsm05_attack_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/FGSM05.csv")
fgsm05_defend_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-FGSM05.csv")
fgsm05_detected_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-FGSM05-detected.csv")

fgsm10_attack_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/FGSM10.csv")
fgsm10_defend_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-FGSM10.csv")
fgsm10_detected_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-FGSM10-detected.csv")

fgsm50_attack_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/FGSM50.csv")
fgsm50_defend_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-FGSM50.csv")
fgsm50_detected_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-FGSM50-detected.csv")

pgd1010_attack_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/PGD1010.csv")
pgd1010_defend_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-PGD1010.csv")
pgd1010_detected_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-PGD1010-detected.csv")

pgd2010_attack_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/PGD2010.csv")
pgd2010_defend_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-PGD2010.csv")
pgd2010_detected_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-PGD2010-detected.csv")

pgd5050_attack_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/PGD5050.csv")
pgd5050_defend_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-PGD5050.csv")
pgd5050_detected_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-PGD5050-detected.csv")

jitter1010_attack_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/Jitter1010.csv")
jitter1010_defend_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-Jitter1010.csv")
jitter1010_detected_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-Jitter1010-detected.csv")

test_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/test.csv")
test_defended_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-test.csv")
test_detected_df = pd.read_csv("/home/grads/hassledw/StyleCLIP_Defense/CelebA_HQ-Labeled/StyleCLIP-test-detected.csv")

## Defense Metrics

In [69]:
def print_test_metrics(test_df, test_defended_df, test_detected_df):
   test_defended_df["predicted_label_test"] = test_df["predicted_label"]
   
   face_detection = pd.merge(test_detected_df, test_defended_df, on=['image'], how='inner').drop(columns=['Unnamed: 0_x', 'image_path_x', 'Unnamed: 0_y', 'image_path_y',
   'actual_label', 'predicted_label', 'confidence'])

   defense_df = pd.merge(test_detected_df, test_defended_df, on=['image'], how='outer')
   defense_df = defense_df[defense_df.isna().any(axis=1)].drop(columns=['Unnamed: 0_x', 'image_path_x', 'Unnamed: 0_y'])
   defense_df = defense_df.rename(columns={'image_path_y': 'image_path'})

   accuracy_orig_df = test_df[test_df["actual_label"] == test_df["predicted_label"]]   
   accuracy_df = defense_df[defense_df["predicted_label_test"] == defense_df["predicted_label"]]
   
   total_images = len(test_df)
   total_defendable_images = len(defense_df)

   accuracy = len(accuracy_df) / total_defendable_images * 100
   accuracy_orig = len(accuracy_orig_df) / total_images * 100
   # accuracy on classifier
   print(f"Accuracy of classifier on original test set {accuracy_orig:.2f}%")
   # A = C
   print(f"Accuracy on StyleCLIP-test {accuracy:.2f}%")


   # img 19 = acutal_label: 5
   # classifier(img 19) = predicted_label : 20

   # classifier(styleClIP(img 19)) = predicted_label : 20
   # compare(styleCLIP(img 19), predicted_label)
   # - does 20 = 20? 

In [70]:
print_test_metrics(test_df, test_defended_df, test_detected_df)

Accuracy of classifier on original test set 81.41%
Accuracy on StyleCLIP-test 68.53%


### Defense Metrics w/o classifier errors on original test set

In [55]:
def print_defense_metrics(attackname, attack_df, defense_df, detected_df, test_df=test_df, use_predicted=True):
   total_images = len(attack_df)

   # Add predicted_labels of the test set to adjust for innate classifier errors since classifier is only 80% accurate.
   attack_df["predicted_label_test"] = test_df["predicted_label"]
   defense_df["predicted_label_test"] = test_df["predicted_label"]
   
   label_comp = "actual_label"
   if use_predicted:
      label_comp = "predicted_label_test"
   
   # Attacked image detection with StyleCLIP.
   face_detection = pd.merge(detected_df, defense_df, on=['image'], how='inner').drop(columns=['Unnamed: 0_x', 'image_path_x', 'Unnamed: 0_y', 'image_path_y',
       'actual_label', 'predicted_label', 'confidence'])
   
   # Filter out face undetected images in defended_df dataset.
   defense_df = pd.merge(detected_df, defense_df, on=['image'], how='outer')
   defense_df = defense_df[defense_df.isna().any(axis=1)].drop(columns=['Unnamed: 0_x', 'image_path_x', 'Unnamed: 0_y'])
   defense_df = defense_df.rename(columns={'image_path_y': 'image_path'})

   # Successful attacks
   attack_result = attack_df[attack_df[label_comp] != attack_df["predicted_label"]]

   # Successful defends inclusive of unsuccessful misclassfication in the attacks.
   defense_result = defense_df[defense_df[label_comp] == defense_df["predicted_label"]]

   # Successful attacks and defenses joined. _x data is the attack, _y is the defense.
   attack_defense_join = pd.merge(attack_result, defense_df, on=['image'], how='inner')

   # Successfully reclassified images after successful attack.
   attack_defense_success = attack_defense_join[attack_defense_join[f"{label_comp}_x"] == attack_defense_join["predicted_label_y"]]
   
   # Instances when the defense was too weak. When a successful attack happened, the defense generated an image with the same label as the attack.
   attack_defense_weak = attack_defense_join[attack_defense_join["predicted_label_x"] == attack_defense_join["predicted_label_y"]]

   # StyleCLIP noticeable classification change. If an attacked image is classified as something else by StyleCLIP, this shows the image is attacked.
   class_change = pd.merge(attack_df, defense_df, on=['image'], how='inner').drop(columns=['Unnamed: 0', 'image_path_x', 'actual_label_x',
        'confidence_x', 'image_path_y', 'actual_label_y', 'confidence_y'])
   class_change = class_change[class_change['predicted_label_x'] != class_change['predicted_label_y']]
   
   attack_effectiveness = len(attack_result) / len(attack_df) * 100
   styleclip_san_all = len(defense_result) / len(defense_df) * 100
   styleclip_san_det_attack = len(attack_defense_success) / len(attack_defense_join) * 100

   attack_face_detected_styleclip = len(face_detection) / total_images * 100
   attack_class_change_styleclip = len(class_change) /total_images * 100
   attack_detected_styleclip = (len(face_detection) + len(class_change)) / total_images * 100

   styleclip_miss = 100 - (len(defense_result) / len(defense_df) * 100)
   defense_too_weak = len(attack_defense_weak) / len(attack_defense_join) * 100
   retention = (attack_detected_styleclip + len(defense_result)) / total_images * 100
   styleclip_net_gain = retention - (100 - attack_effectiveness)

   print("-" * 50)
   print(f"RUNNING {attackname} DEFENSE METRICS...\n")
   print(f"Attack Effectiveness: {attack_effectiveness:.2f}%\n")

   # A = C
   print(f"StyleCLIP Sanitization Effectiveness (on ALL defendable images): {styleclip_san_all:.2f}%")
   # A != B and A = C
   print(f"StyleCLIP Sanitization Effectiveness (on successful attacks): {styleclip_san_det_attack:.2f}%")

   # Face detect, shape error thrown, stored in detected_df.
   # original classification: A = C(A)
   # classification change: B != C
   print()
   print(f"Attacked Image Detection w/ StyleCLIP: {attack_detected_styleclip:.2f}%")
   print(f"\t Couldn't Detect a Face: {attack_face_detected_styleclip:.2f}%")
   print(f"\t Classification Change: {attack_class_change_styleclip:.2f}%")
   print()

   # A != C
   print(f"Misclassification on StyleCLIP Images: {styleclip_miss:.2f}%")
   # A != B and B = C
   print(f"Successful Attacks Where Sanitization TOO Weak: {defense_too_weak:.2f}%")

   # print(f"Total StyleCLIP Dataset Retention: {retention:.2f}%")
   
   # (sum(defended) / total_images) - (100 - attack_effectiveness)
   print(f"Net Gain Using StyleCLIP as a Defense: {styleclip_net_gain:.2f}%")
   print("-" * 50)

In [56]:
print_defense_metrics("FGSM05", fgsm05_attack_df, fgsm05_defend_df, fgsm05_detected_df)
print_defense_metrics("FGSM10", fgsm10_attack_df, fgsm10_defend_df, fgsm10_detected_df)
print_defense_metrics("FGSM50", fgsm50_attack_df, fgsm50_defend_df, fgsm50_detected_df)

print_defense_metrics("PGD1010", pgd1010_attack_df, pgd1010_defend_df, pgd1010_detected_df)
print_defense_metrics("PGD2010", pgd2010_attack_df, pgd2010_defend_df, pgd2010_detected_df)
print_defense_metrics("PGD5050", pgd5050_attack_df, pgd5050_defend_df, pgd5050_detected_df)

print_defense_metrics("Jitter1010", jitter1010_attack_df, jitter1010_defend_df, jitter1010_detected_df)

--------------------------------------------------
RUNNING FGSM05 DEFENSE METRICS...

Attack Effectiveness: 19.10%

StyleCLIP Sanitization Effectiveness (on ALL defendable images): 51.78%
StyleCLIP Sanitization Effectiveness (on successful attacks): 10.53%

Attacked Image Detection w/ StyleCLIP: 42.71%
	 Couldn't Detect a Face: 1.01%
	 Classification Change: 41.71%

Misclassification on StyleCLIP Images: 48.22%
Successful Attacks Where Sanitization TOO Weak: 42.11%
Net Gain Using StyleCLIP as a Defense: -8.18%
--------------------------------------------------
--------------------------------------------------
RUNNING FGSM10 DEFENSE METRICS...

Attack Effectiveness: 46.73%

StyleCLIP Sanitization Effectiveness (on ALL defendable images): 30.61%
StyleCLIP Sanitization Effectiveness (on successful attacks): 7.69%

Attacked Image Detection w/ StyleCLIP: 50.25%
	 Couldn't Detect a Face: 1.51%
	 Classification Change: 48.74%

Misclassification on StyleCLIP Images: 69.39%
Successful Attacks 